Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

C:\Users\Matt\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [5]:
num_steps = 1801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 33.747356
Training accuracy: 10.6%
Validation accuracy: 18.0%
Loss at step 100: 3.624777
Training accuracy: 73.8%
Validation accuracy: 72.2%
Loss at step 200: 2.709112
Training accuracy: 76.1%
Validation accuracy: 73.2%
Loss at step 300: 2.229311
Training accuracy: 77.3%
Validation accuracy: 73.5%
Loss at step 400: 1.916582
Training accuracy: 78.0%
Validation accuracy: 73.8%
Loss at step 500: 1.688919
Training accuracy: 78.8%
Validation accuracy: 74.0%
Loss at step 600: 1.513106
Training accuracy: 79.5%
Validation accuracy: 73.9%
Loss at step 700: 1.372640
Training accuracy: 80.2%
Validation accuracy: 73.9%
Loss at step 800: 1.257708
Training accuracy: 80.8%
Validation accuracy: 73.8%
Loss at step 900: 1.161835
Training accuracy: 81.5%
Validation accuracy: 73.8%
Loss at step 1000: 1.080622
Training accuracy: 82.0%
Validation accuracy: 73.8%
Loss at step 1100: 1.010991
Training accuracy: 82.5%
Validation accuracy: 73.9%
Loss at step 1200: 0.950719
Training ac

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [6]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 34.056267
Minibatch accuracy: 9.4%
Validation accuracy: 14.4%
Minibatch loss at step 500: 1.868928
Minibatch accuracy: 79.7%
Validation accuracy: 74.6%
Minibatch loss at step 1000: 1.779164
Minibatch accuracy: 77.3%
Validation accuracy: 75.3%
Minibatch loss at step 1500: 1.209214
Minibatch accuracy: 82.0%
Validation accuracy: 76.0%
Minibatch loss at step 2000: 1.391909
Minibatch accuracy: 79.7%
Validation accuracy: 74.9%
Minibatch loss at step 2500: 1.551426
Minibatch accuracy: 71.9%
Validation accuracy: 76.4%
Minibatch loss at step 3000: 1.511723
Minibatch accuracy: 76.6%
Validation accuracy: 78.4%
Test accuracy: 85.4%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

Rewrite $x * W_1 + b_1 = x' * W_1'$ really think of this as $\sum X_k W_k + b_k$ where $k = 0$ and $X_0 = 1$

Example from [vinhkhuc](https://gist.github.com/vinhkhuc/e53a70f9e5c3f55852b0)

---

In [8]:
# Start off with the same code from SGD
hidden_size = 1024 
input_size = image_size * image_size + 1
output_size = 10


def init_weights(shape):
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)


def forwardprop(X, w_1, w_2):
    """
    yhat is not softmax since Tensorflows softmax_cross_entropy_with_logits does that
    """
    # convert this to use relu instead of sigmoid
    h = tf.nn.sigmoid(tf.matmul(X,w_1)) # the \sigma function
    #h = tf.nn.relu(tf.matmul(X,w_1)) # the \sigma function
    yhat = tf.matmul(h, w_2) # the \varphi function
    return yhat



graph = tf.Graph()
with graph.as_default():

    X = tf.placeholder(tf.float32, shape=[None, image_size * image_size + 1])
    y = tf.placeholder(tf.float32, shape=[None, num_labels])
    #tf_valid_dataset = tf.constant(valid_dataset)
    #tf_test_dataset = tf.constant(test_dataset)
    
    
    w1 = init_weights((input_size, hidden_size))
    w2 = init_weights((hidden_size, output_size))

    # forward propagation
    yhat = forwardprop(X, w1, w2)
    predict = tf.argmax(yhat, axis=1)
    
    # backward propagation (cost)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
    #  this seems to work, but I'm not sure where predict comes into play here
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    



We are now ready to run this


In [9]:
# this took a LOOONG time to run

num_steps = 200
train_subset = 20000
batch_size = 256

train_X = np.ones((train_subset, input_size))
train_X[:, 1:] = train_dataset[:train_subset, :] 
train_y =  train_labels[:train_subset]

test_X = np.ones((test_dataset.shape[0], input_size))
test_X[:, 1:] = test_dataset
test_y = test_labels

valid_X = np.ones((valid_dataset.shape[0], input_size))
valid_X[:, 1:] = valid_dataset
valid_y = valid_labels

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    print("Initialized")
    # this is the epoch steps
    for step in range(num_steps):
        
        # train with one example each time
        #for i in np.arange(0,len(train_X),2):
        #    session.run(optimizer,feed_dict = {X : train_X[i: i+2], y: train_y[i:i+2]})

        # train with a batch of 128 at a time
        offset = (step * batch_size) % (train_y.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_X[offset:(offset + batch_size), :]
        batch_labels = train_y[offset:(offset + batch_size), :]
        session.run(optimizer, feed_dict = {X : batch_data, y : batch_labels})
            
            
            
        train_acc = np.mean(np.argmax(train_y,axis=1)==session.run(predict, feed_dict={X:train_X, y: train_y}))
        test_acc = np.mean(np.argmax(test_y,axis=1)==session.run(predict, feed_dict={X:test_X, y: test_y}))
        valid_acc = np.mean(np.argmax(valid_y,axis=1)==session.run(predict, feed_dict={X:valid_X, y: valid_y}))
        
        print("Epoch = %2d, training accuracy = %.2f%%, test accuracy = %.2f%%, valid accuracy = %.2f%%" % (step, 100. *train_acc, 100. * test_acc, 100. * valid_acc))




Initialized
Epoch =  0, training accuracy = 13.48%, test accuracy = 13.50%, valid accuracy = 12.96%
Epoch =  1, training accuracy = 28.87%, test accuracy = 31.76%, valid accuracy = 29.10%
Epoch =  2, training accuracy = 23.70%, test accuracy = 24.90%, valid accuracy = 23.45%
Epoch =  3, training accuracy = 37.10%, test accuracy = 41.04%, valid accuracy = 36.95%
Epoch =  4, training accuracy = 40.45%, test accuracy = 44.32%, valid accuracy = 40.19%
Epoch =  5, training accuracy = 40.28%, test accuracy = 44.09%, valid accuracy = 40.48%
Epoch =  6, training accuracy = 43.73%, test accuracy = 46.37%, valid accuracy = 42.92%
Epoch =  7, training accuracy = 45.77%, test accuracy = 50.18%, valid accuracy = 45.34%
Epoch =  8, training accuracy = 52.80%, test accuracy = 57.71%, valid accuracy = 52.42%
Epoch =  9, training accuracy = 55.02%, test accuracy = 60.47%, valid accuracy = 55.21%
Epoch = 10, training accuracy = 58.72%, test accuracy = 64.16%, valid accuracy = 58.40%
Epoch = 11, training

Epoch = 93, training accuracy = 80.05%, test accuracy = 86.31%, valid accuracy = 78.62%
Epoch = 94, training accuracy = 80.17%, test accuracy = 86.47%, valid accuracy = 79.16%
Epoch = 95, training accuracy = 80.77%, test accuracy = 87.06%, valid accuracy = 79.84%
Epoch = 96, training accuracy = 80.73%, test accuracy = 87.01%, valid accuracy = 79.99%
Epoch = 97, training accuracy = 81.07%, test accuracy = 87.42%, valid accuracy = 80.03%
Epoch = 98, training accuracy = 80.38%, test accuracy = 86.88%, valid accuracy = 79.57%
Epoch = 99, training accuracy = 80.59%, test accuracy = 86.78%, valid accuracy = 79.49%
Epoch = 100, training accuracy = 81.09%, test accuracy = 87.48%, valid accuracy = 80.16%
Epoch = 101, training accuracy = 80.95%, test accuracy = 87.33%, valid accuracy = 80.01%
Epoch = 102, training accuracy = 81.10%, test accuracy = 87.42%, valid accuracy = 80.09%
Epoch = 103, training accuracy = 80.91%, test accuracy = 87.42%, valid accuracy = 80.17%
Epoch = 104, training accura

Epoch = 186, training accuracy = 82.56%, test accuracy = 88.39%, valid accuracy = 81.17%
Epoch = 187, training accuracy = 81.85%, test accuracy = 87.86%, valid accuracy = 80.47%
Epoch = 188, training accuracy = 81.97%, test accuracy = 87.94%, valid accuracy = 80.88%
Epoch = 189, training accuracy = 82.16%, test accuracy = 87.98%, valid accuracy = 80.94%
Epoch = 190, training accuracy = 81.75%, test accuracy = 87.77%, valid accuracy = 80.68%
Epoch = 191, training accuracy = 82.06%, test accuracy = 88.28%, valid accuracy = 80.82%
Epoch = 192, training accuracy = 82.34%, test accuracy = 88.24%, valid accuracy = 81.05%
Epoch = 193, training accuracy = 82.16%, test accuracy = 88.29%, valid accuracy = 81.24%
Epoch = 194, training accuracy = 82.38%, test accuracy = 88.11%, valid accuracy = 81.16%
Epoch = 195, training accuracy = 82.50%, test accuracy = 88.25%, valid accuracy = 81.17%
Epoch = 196, training accuracy = 82.38%, test accuracy = 88.15%, valid accuracy = 81.04%
Epoch = 197, training